# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [3]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Generated Code

In [5]:
import base64
import os
from google import genai
from google.genai import types


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro-preview-05-06"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""Ta tâche est d’évaluer la proximité sémantique entre le sujet suivant :
**« Comment connaît-on son devoir ? »**
 et chacune des 17 notions philosophiques suivantes (avec leur identifiant en réponse de sortie) :
*   1: l’art
*   2: le bonheur
*   3: la conscience
*   4: le devoir
*   5: l’État
*   6: l’inconscient
*   7: la justice
*   8: le langage
*   9: la liberté
*   10: la nature
*   11: la raison
*   12: la religion
*   13: la science
*   14: la technique
*   15: le temps
*   16: le travail
*   17: la vérité

**Instructions :**
Pour chacune des 17 notions listées ci-dessus :
1.  Évaluez dans quelle mesure elle est susceptible d’être mobilisée de manière pertinente pour traiter le sujet : « Comment connaît-on son devoir ? ».
2.  Attribuez une note de pertinence sur 10 (où 0 signifie 'aucune pertinence' et 10 signifie 'pertinence maximale / lien conceptuel extrêmement fort'). Cette note indique son degré de proximité conceptuelle avec le sujet.

**Format de Réponse Attendu :**
Répondez **uniquement** avec un objet JSON. La structure de cet objet doit être la suivante :
La clé `sujet` doit contenir la chaîne de caractères du sujet.
Les clés suivantes doivent être les identifiants numériques des notions (de "1" à "17", sous forme de chaînes de caractères).
La valeur associée à chaque clé numérique doit être **uniquement la note de pertinence** (un nombre entier entre 0 et 10). Aucune justification ni aucun autre texte ne doit être fourni pour chaque notion.

**Exemple de format :**
```json
{
  "sujet": "Comment connaît-on son devoir ?",
  "1": 2,   // Note pour l'art
  "2": 5,   // Note pour le bonheur
  "3": 9,   // Note pour la conscience
  // ...et ainsi de suite jusqu'à 17...
  "17": 7  // Note pour la vérité
}
```
Assurez-vous que les 17 notions sont évaluées et présentes dans la réponse JSON avec leurs identifiants numériques respectifs comme clés, et que la valeur associée à chaque notion est **exclusivement sa note numérique**."""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        system_instruction=[
            types.Part.from_text(text="""Vous devez évaluer la proximité sémantique entre le sujet de dissertation de philosophie donné et les 17 notions philosophiques données."""),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()


{
  "sujet": "Comment connaît-on son devoir ?",
  "1": 3,
  "2": 5,
  "3": 9,
  "4": 10,
  "5": 7,
  "6": 3,
  "7": 8,
  "8": 6,
  "9": 7,
  "10": 5,
  "11": 9,
  "12": 8,
  "13": 2,
  "14": 2,
  "15": 3,
  "16": 4,
  "17": 7
}

In [ ]:
import csv
import json
import os
import pandas as pd
from google import genai
from google.genai import types # type: ignore # Ignore pour type: ignore car genai.types peut être dynamique

# --- Configuration de la clé API ---
# Essayez d'abord les secrets de Colab, puis les variables d'environnement.
# Vous DEVEZ configurer votre clé API pour que cela fonctionne.
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if GEMINI_API_KEY:
        os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
    else:
        print("Clé API GEMINI non trouvée dans les secrets Colab. Vérifiez la variable d'environnement.")
except ImportError:
    print("Non exécuté dans Colab, vérification de la variable d'environnement GEMINI_API_KEY.")
    pass # GEMINI_API_KEY devrait déjà être dans os.environ si non Colab

if not os.environ.get("GEMINI_API_KEY"):
    print("ERREUR CRITIQUE: La variable d'environnement GEMINI_API_KEY n'est pas définie.")
    print("Veuillez la configurer dans les secrets de Colab (Insert -> Code snippet -> Secrets) ou dans votre environnement.")
    # Vous pouvez arrêter le script ici si vous le souhaitez, car il échouera de toute façon.
    # exit()


def call_gemini_for_subject(current_subject: str) -> str | None:
    """
    Prend un sujet de dissertation, l'insère dans le prompt et appelle l'API Gemini.
    Retourne la réponse JSON brute sous forme de chaîne ou None en cas d'erreur.
    """
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        print(f"Erreur: Clé API GEMINI non disponible pour le sujet : {current_subject}")
        return None

    client = genai.Client(api_key=api_key)

    model_name = "gemini-2.5-pro-preview-05-06"

    # Le prompt est formaté avec le sujet actuel.
    # Notez l'utilisation de {{ et }} pour échapper les accolades dans l'exemple JSON à l'intérieur de la f-string.
    prompt_text = f"""Ta tâche est d’évaluer la proximité sémantique entre le sujet suivant :
**« {current_subject} »**
 et chacune des 17 notions philosophiques suivantes (avec leur identifiant en réponse de sortie) :
*   1: l’art
*   2: le bonheur
*   3: la conscience
*   4: le devoir
*   5: l’État
*   6: l’inconscient
*   7: la justice
*   8: le langage
*   9: la liberté
*   10: la nature
*   11: la raison
*   12: la religion
*   13: la science
*   14: la technique
*   15: le temps
*   16: le travail
*   17: la vérité

**Instructions :**
Pour chacune des 17 notions listées ci-dessus :
1.  Évaluez dans quelle mesure elle est susceptible d’être mobilisée de manière pertinente pour traiter le sujet : « {current_subject} ».
2.  Attribuez une note de pertinence sur 10 (où 0 signifie 'non pertinent / notion qui non reliée au sujet' et 10 signifie 'pertinence maximale / lien extrêmement fort'). Cette note indique son degré de proximité conceptuelle avec le sujet.

**Format de Réponse Attendu :**
Répondez **uniquement** avec un objet JSON. La structure de cet objet doit être la suivante :
La clé `sujet` doit contenir la chaîne de caractères du sujet.
Les clés suivantes doivent être les identifiants numériques des notions (de "1" à "17", sous forme de chaînes de caractères).
La valeur associée à chaque clé numérique doit être **uniquement la note de pertinence** (un nombre entier entre 0 et 10). Aucune justification ni aucun autre texte ne doit être fourni pour chaque notion.

**Exemple de format :**
```json
{{
  "sujet": "{current_subject}",
  "1": 2,   // Note pour l'art
  "2": 5,   // Note pour le bonheur
  "3": 9,   // Note pour la conscience
  // ...et ainsi de suite jusqu'à 17...
  "17": 7  // Note pour la vérité
}}
```
Assurez-vous que les 17 notions sont évaluées et présentes dans la réponse JSON avec leurs identifiants numériques respectifs comme clés, et que la valeur associée à chaque notion est **exclusivement sa note numérique**."""

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt_text),
            ],
        ),
    ]

    # Configuration telle que fournie dans votre code original
    # Note: La validité de `system_instruction` dans `GenerateContentConfig` peut dépendre
    # de la version du SDK `google-generativeai`. Les versions récentes le placent
    # dans `GenerativeModel(system_instruction=...)` ou comme un message de rôle 'system' dans `contents`.
    # Je conserve votre structure car vous avez indiqué qu'elle fonctionnait.
    try:
        generate_content_config = types.GenerateContentConfig(
            response_mime_type="application/json",
            system_instruction=[ # Conservé tel que dans votre code original
                types.Part.from_text(text="""Vous devez évaluer la proximité sémantique entre le sujet de dissertation de philosophie donné et les 17 notions philosophiques données."""),
            ],
        )
    except TypeError as te:
        # Si cela se produit, c'est que system_instruction n'est pas un paramètre valide pour GenerateContentConfig
        # dans la version actuelle du SDK.
        print(f"Avertissement: Erreur lors de la création de GenerateContentConfig avec system_instruction: {te}")
        print("Cela peut indiquer une incompatibilité avec la version actuelle du SDK google-generativeai.")
        print("Tentative sans system_instruction dans GenerateContentConfig.")
        generate_content_config = types.GenerateContentConfig(
            response_mime_type="application/json"
        )
        # Idéalement, le system_instruction devrait être ajouté à `contents` :
        # system_instruction_part = types.Part.from_text(text="...")
        # contents.insert(0, types.Content(role="system", parts=[system_instruction_part]))
        # Pour l'instant, on continue sans pour respecter au mieux la demande initiale.

    full_response_text = ""
    try:
        response_stream = client.models.generate_content_stream(
            model=model_name, # Utilise le nom du modèle défini ci-dessus
            contents=contents,
            config=generate_content_config,
        )
        for chunk in response_stream:
            if hasattr(chunk, 'text') and chunk.text:
                full_response_text += chunk.text

        if not full_response_text.strip():
            print(f"Avertissement: Réponse vide de l'API pour le sujet '{current_subject}'")
            return None
        return full_response_text

    except Exception as e:
        print(f"Erreur lors de l'appel à l'API Gemini pour le sujet '{current_subject}': {e}")
        if hasattr(e, 'response') and e.response: # type: ignore
            print(f"Détails de l'erreur API: {e.response}") # type: ignore
        return None


def run_batch_processing():
    """
    Fonction principale pour lire les sujets, appeler Gemini, et écrire les résultats.
    """
    input_csv_path = "/content/file.csv"
    output_csv_path = "/content/output.csv"

    # Création d'un fichier d'exemple si file.csv n'existe pas (pour faciliter les tests)
    if not os.path.exists(input_csv_path):
        print(f"Fichier '{input_csv_path}' non trouvé. Création d'un fichier d'exemple.")
        example_data = [
            ["Le bonheur est-il une affaire privée ?"],
            ["L'art nous détourne-t-il du réel ?"],
            ["Peut-on se mentir à soi-même ?"],
            ["La technique ne pose-t-elle que des problèmes moraux ?"],
            [""] # Ligne vide pour tester la robustesse
        ]
        try:
            pd.DataFrame(example_data).to_csv(input_csv_path, header=False, index=False)
            print(f"Fichier d'exemple '{input_csv_path}' créé avec {len(example_data)-1} sujets.")
        except Exception as e:
            print(f"Impossible de créer le fichier d'exemple : {e}")
            return

    try:
        df_input = pd.read_csv(input_csv_path, header=None, usecols=[0], dtype=str)
        # Supprimer les lignes vides ou ne contenant que des espaces
        subjects = df_input.iloc[:, 0].str.strip().dropna()
        subjects = subjects[subjects != ''].tolist()

    except FileNotFoundError:
        print(f"Erreur : Le fichier d'entrée '{input_csv_path}' n'a pas été trouvé.")
        return
    except pd.errors.EmptyDataError:
        print(f"Erreur : Le fichier d'entrée '{input_csv_path}' est vide ou ne contient pas la colonne attendue.")
        return
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier CSV '{input_csv_path}': {e}")
        return

    if not subjects:
        print(f"Aucun sujet de dissertation trouvé dans '{input_csv_path}'.")
        return

    print(f"{len(subjects)} sujets de dissertation à traiter.")

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as outfile:
        csv_writer = csv.writer(outfile)
        # Écriture de l'en-tête
        header = ["Sujet"] + [str(i) for i in range(1, 18)] # "Sujet", "1", "2", ..., "17"
        csv_writer.writerow(header)

        for i, subject_text in enumerate(subjects):
            print(f"\nTraitement du sujet {i+1}/{len(subjects)}: \"{subject_text}\"")

            json_response_str = call_gemini_for_subject(subject_text)

            if json_response_str:
                try:
                    # Nettoyer la réponse si elle contient des ```json ... ``` autour
                    if json_response_str.strip().startswith("```json"):
                        json_response_str = json_response_str.strip()[7:]
                    if json_response_str.strip().endswith("```"):
                        json_response_str = json_response_str.strip()[:-3]

                    data = json.loads(json_response_str.strip())

                    # Vérification (optionnelle) que le sujet dans la réponse correspond
                    # if data.get("sujet") != subject_text:
                    #     print(f"  Avertissement: Discordance de sujet. Attendu: '{subject_text}', Reçu: '{data.get('sujet')}'")

                    scores = [data.get(str(j), "ERR_SCORE") for j in range(1, 18)]
                    csv_writer.writerow([subject_text] + scores)
                    print(f"  Scores enregistrés pour : \"{subject_text}\"")

                except json.JSONDecodeError:
                    print(f"  Erreur : Impossible de décoder la réponse JSON pour le sujet '{subject_text}'.")
                    print(f"  Réponse brute reçue : {json_response_str}")
                    csv_writer.writerow([subject_text] + ["JSON_DECODE_ERROR"] * 17)
                except KeyError as e:
                    print(f"  Erreur : Clé manquante {e} dans la réponse JSON pour le sujet '{subject_text}'.")
                    csv_writer.writerow([subject_text] + ["MISSING_KEY_ERROR"] * 17)
                except Exception as e:
                    print(f"  Erreur inattendue lors du traitement de la réponse pour '{subject_text}': {e}")
                    csv_writer.writerow([subject_text] + ["PROCESSING_ERROR"] * 17)
            else:
                print(f"  Échec de l'obtention d'une réponse pour le sujet '{subject_text}'.")
                csv_writer.writerow([subject_text] + ["API_CALL_FAILED"] * 17)

            # Petite pause pour éviter de surcharger l'API (si nécessaire)
            # import time
            # time.sleep(1) # 1 seconde entre les requêtes

    print(f"\nTraitement terminé. Les résultats sont sauvegardés dans '{output_csv_path}'.")

if __name__ == "__main__":
    # S'assurer que la clé API est disponible avant de lancer le traitement
    if os.environ.get("GEMINI_API_KEY"):
        run_batch_processing()
    else:
        print("Le traitement par lots n'a pas été lancé car GEMINI_API_KEY n'est pas configurée.")
        print("Veuillez vérifier la configuration de votre clé API (secrets Colab ou variable d'environnement).")

90 sujets de dissertation à traiter.

Traitement du sujet 1/90: "La pluralité des cultures fait-elle obstacle à l’unité du genre humain ?"
  Scores enregistrés pour : "La pluralité des cultures fait-elle obstacle à l’unité du genre humain ?"

Traitement du sujet 2/90: "Reconnaître ses devoirs, est-ce renoncer à sa liberté ?"
  Scores enregistrés pour : "Reconnaître ses devoirs, est-ce renoncer à sa liberté ?"

Traitement du sujet 3/90: "Seul ce qui peut s'échanger a-t-il de la valeur ?"
  Scores enregistrés pour : "Seul ce qui peut s'échanger a-t-il de la valeur ?"

Traitement du sujet 4/90: "Les lois peuvent-elles faire notre bonheur ?"
  Scores enregistrés pour : "Les lois peuvent-elles faire notre bonheur ?"

Traitement du sujet 5/90: "Sommes-nous conscients de ce que nous désirons ?"
  Scores enregistrés pour : "Sommes-nous conscients de ce que nous désirons ?"

Traitement du sujet 6/90: "La technique nous libère-t-elle ?"
  Scores enregistrés pour : "La technique nous libère